## Importes de bibliotecas

In [23]:
from qiskit_nature.units import *   #ANGSTROM
from qiskit_nature.second_q.drivers import *    #PySCFDriver
from qiskit_nature.second_q.mappers import * #JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.problems import EigenstateResult
from qiskit_nature.second_q.circuit.library import *  #Ansatz, HF
#from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator, Options
from qiskit_algorithms.optimizers import * 
from qiskit_algorithms import VQE
from braket.tracking import Tracker
from qiskit_braket_provider import *
from qiskit_aer import StatevectorSimulator
from qiskit.primitives import Estimator
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

## Mapeamentos de operaadores para o $H_2$

In [2]:
driver = PySCFDriver(atom=f"H 0 0 0; H 0 0 0.7", basis="sto-3g")
estrutura = driver.run() #Criamos a estrutura do nosso sistema

#obtemos os operadores fermionicos que são usados na segunda quantização do nosso problema.
operadores = estrutura.hamiltonian.second_q_op()

In [3]:
#mapeadores que estamos utilizando
mapas = [JordanWignerMapper(), BravyiKitaevMapper()]

#mapeadores depois de reduzidos por simetria
mapas_temperados = [estrutura.get_tapered_mapper(aux) for aux in mapas]

In [4]:
#Temos aqui a configuração dos nossos operadores fermionicos
print(operadores)

#quantos termos de operadores temos:
print(len(operadores))

Fermionic Operator
number spin orbitals=4, number terms=36
  0.34119476657602105 * ( +_0 +_0 -_0 -_0 )
+ 0.3353663891543795 * ( +_0 +_1 -_1 -_0 )
+ 0.34119476657602105 * ( +_0 +_2 -_2 -_0 )
+ 0.3353663891543795 * ( +_0 +_3 -_3 -_0 )
+ 0.08950028803070331 * ( +_0 +_0 -_1 -_1 )
+ 0.08950028803070331 * ( +_0 +_1 -_0 -_1 )
+ 0.08950028803070331 * ( +_0 +_2 -_3 -_1 )
+ 0.08950028803070331 * ( +_0 +_3 -_2 -_1 )
+ 0.08950028803070331 * ( +_1 +_0 -_1 -_0 )
+ 0.08950028803070331 * ( +_1 +_1 -_0 -_0 )
+ 0.08950028803070331 * ( +_1 +_2 -_3 -_0 )
+ 0.08950028803070331 * ( +_1 +_3 -_2 -_0 )
+ 0.3353663891543795 * ( +_1 +_0 -_0 -_1 )
+ 0.35255281608639233 * ( +_1 +_1 -_1 -_1 )
+ 0.3353663891543795 * ( +_1 +_2 -_2 -_1 )
+ 0.35255281608639233 * ( +_1 +_3 -_3 -_1 )
+ 0.34119476657602105 * ( +_2 +_0 -_0 -_2 )
+ 0.3353663891543795 * ( +_2 +_1 -_1 -_2 )
+ 0.34119476657602105 * ( +_2 +_2 -_2 -_2 )
+ 0.3353663891543795 * ( +_2 +_3 -_3 -_2 )
+ 0.08950028803070331 * ( +_2 +_0 -_1 -_3 )
+ 0.08950028803070331 *

## Visualização dos operadores que são usados em PauliStrings

In [5]:
#Estamos mapeando os nossos operadores fermionicos para o QubitSpace:
circuitos = [aux.map(operadores) for aux in mapas]

observaveis = []
observaveis.append([circuitos[0].to_list()[i][0] for i in range(len(circuitos[0]))])
observaveis.append([circuitos[1].to_list()[i][0] for i in range(len(circuitos[0]))])

obs_jw = ' + '.join(observaveis[0])
obs_bk = ' + '.join(observaveis[1])

print(obs_bk)
print(obs_jw)

#mapeamento utilizando os mapas tapered
circuitos_temperados = [aux.map(operadores) for aux in mapas_temperados]

print()
print(circuitos_temperados[0])
print(circuitos_temperados[1])


IIII + IIIZ + IIZZ + IIZI + IZII + IZIZ + ZZZI + ZZZZ + ZXIX + IXZX + ZXZX + IXIX + IZZZ + ZZIZ + ZIZI
IIII + IIIZ + IIZI + IIZZ + IZII + IZIZ + ZIII + ZIIZ + YYYY + XXYY + YYXX + XXXX + IZZI + ZIZI + ZZII

SparsePauliOp(['I', 'Z', 'X'],
              coeffs=[-1.03240512+0.j, -0.84091136+0.j,  0.17900058+0.j])
SparsePauliOp(['I', 'Z', 'X'],
              coeffs=[-1.03240512+0.j,  0.84091136+0.j,  0.17900058+0.j])


## Funções para H2 Padrão e Tapered

In [14]:
def h2_default(d, mapa, otimizador = SLSQP()):
    driver = PySCFDriver(atom=f"H 0 0 0; H 0 0 {d}", basis="sto-3g")
    estrutura = driver.run() #Criamos a estrutura do nosso sistema

    ansatz = UCCSD(
        estrutura.num_spatial_orbitals,
        estrutura.num_particles,
        mapa,
        initial_state=HartreeFock(
            estrutura.num_spatial_orbitals,
            estrutura.num_particles,
            mapa))

    vqe_solver = VQE(Estimator(), ansatz, otimizador)
    vqe_solver.initial_point = [0.0] * ansatz.num_parameters
    calculo = GroundStateEigensolver(mapa, vqe_solver)
    resultados = calculo.solve(estrutura)
    return resultados.groundenergy + resultados.nuclear_repulsion_energy


In [18]:
def h2_tapered(d, mapa, otimizador = SLSQP()):
    driver = PySCFDriver(atom=f"H 0 0 0; H 0 0 {d}", basis="sto-3g")
    estrutura = driver.run() #Criamos a estrutura do nosso sistema

    op = estrutura.get_tapered_mapper(mapa)

    ansatz = UCCSD(estrutura.num_spatial_orbitals,
            estrutura.num_particles,
            op,
            initial_state=HartreeFock(
                estrutura.num_spatial_orbitals,
                estrutura.num_particles,
                op))

    vqe_solver = VQE(Estimator(), ansatz, otimizador)
    vqe_solver.initial_point = [0.0] * ansatz.num_parameters
    calculo = GroundStateEigensolver(op, vqe_solver)
    resultados = calculo.solve(estrutura)
    return resultados.groundenergy + resultados.nuclear_repulsion_energy


## Compração dos Resultados

In [11]:
intervalo = np.concatenate((np.linspace(0.2,2,15), np.linspace(2.2,6,5)))

In [36]:
%%timeit
h2_def_jw = [h2_default(d, JordanWignerMapper()) for d in intervalo]

27.4 s ± 257 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit
h2_def_bk = [h2_default(d, BravyiKitaevMapper()) for d in intervalo]

23.1 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%%timeit
h2_tap = [h2_tapered(d, BravyiKitaevMapper()) for d in intervalo]

8.15 s ± 154 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
a = [h2_tap[i] - (h2_def_bk[i] + h2_def_jw[i])/2 for i in range(len(h2_tap))] 
np.mean(a)

6.185774115152753e-13